*   nama : rizky adhi nugroho
*   domisili : tangerang selatan
*   email : nugrohorizkyadhi@gmail.com




import data dari kaggle

In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rizkyadhiiii","key":"8ad012eb3b08719fd1db8a4ac7618a95"}'}

In [ ]:
#download dari kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d harmanpreet93/hotelreviews

 40% 5.00M/12.4M [00:00<00:00, 26.2MB/s]
100% 12.4M/12.4M [00:00<00:00, 38.3MB/s]


In [ ]:
#ekstrak data
! unzip /content/hotelreviews.zip

Archive:  /content/hotelreviews.zip
  inflating: hotel-reviews.csv       


membuat dataframe

In [ ]:
import pandas as pd
df = pd.read_csv('/content/hotel-reviews.csv')
df = df.drop(df.columns[[0,2,3]], axis=1)

In [ ]:
df.tail()

,Description,Is_Response
38927,We arrived late at night and walked in to a ch...,happy
38928,The only positive impression is location and p...,not happy
38929,Traveling with friends for shopping and a show...,not happy
38930,The experience was just ok. We paid extra for ...,not happy
38931,The Westin is a wonderfully restored grande da...,happy


In [ ]:
category = pd.get_dummies(df.Is_Response)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='Is_Response')
df_baru

,Description,happy,not happy
0,The room was kind of clean but had a VERY stro...,0,1
1,I stayed at the Crown Plaza April -- - April -...,0,1
2,I booked this hotel through Hotwire at the low...,0,1
3,Stayed here with husband and sons on the way t...,1,0
4,My girlfriends and I stayed here to celebrate ...,0,1
...,...,...,...
38927,We arrived late at night and walked in to a ch...,1,0
38928,The only positive impression is location and p...,0,1
38929,Traveling with friends for shopping and a show...,0,1
38930,The experience was just ok. We paid extra for ...,0,1


In [ ]:
deskripsi = df_baru['Description'].values
label = df_baru[['happy','not happy']].values

split data 80 20

In [ ]:
from sklearn.model_selection import train_test_split
deskripsi_latih, deskripsi_test, label_latih, label_test = train_test_split(deskripsi, label, test_size=0.2)

tokenisasi

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(deskripsi_latih) 
tokenizer.fit_on_texts(deskripsi_test)
 
sekuens_latih = tokenizer.texts_to_sequences(deskripsi_latih)
sekuens_test = tokenizer.texts_to_sequences(deskripsi_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

membuat arsitektur model

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=500, output_dim=16), #menggunakan Embedding
    tf.keras.layers.LSTM(32), #menggunakan LSTM
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dropout(0.2), #dropout untuk mencegah overfit
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5), #dropout untuk mencegah overfit
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          8000      
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6272      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
_________________________________________________________________
dropout (Dropout)            (None, 12)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 24)                312       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 5

callback

In [ ]:
#callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.85 and logs.get('val_accuracy')>0.85):
      print("\n accurasi dan val_accuracy telah mencapai >85%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
num_epochs = 15
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), batch_size=64,
                    callbacks=[callbacks], verbose=2)

Epoch 1/15
487/487 - 106s - loss: 0.4600 - accuracy: 0.7872 - val_loss: 0.3480 - val_accuracy: 0.8481
Epoch 2/15
487/487 - 101s - loss: 0.3686 - accuracy: 0.8517 - val_loss: 0.3364 - val_accuracy: 0.8558

 accurasi dan val_accuracy telah mencapai >85%!
